<div align="center">

# **Alejandro Coman Venceslá**

### Doble Grado en Ingeniería Informática y  
### Administración y Dirección de Empresas
#### Universidad de Granada

<br>

<div align="center">
  <img src="https://etsiit.ugr.es/sites/centros/etsiit/public/template-extra/etsiit-logo.png" alt="Imagen 1" style="width: 200px; margin-right: 40px;">
  <img src="https://etsiit.ugr.es/sites/centros/etsiit/public/color/ugr-41cc9222/logo-mono.svg" alt="Imagen 2" style="width: 300px; margin-left: 40px; margin-bottom: 60px">
</div>

**Trabajo de Fin de Grado**

<br><br>

*Análisis de sesgos en modelos de inteligencia artificial generativa textual.*

</div>

# Capítulo 2.2. Extracción de datos de las biografías

Este cuaderno contiene el código empleado para el primer capítulo 

In [55]:
# 1. Librerías necesarias
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import sys
import os
import re

# Para TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Para sentiment analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

# 1. Calcula la carpeta padre del notebook
parent_dir = Path().resolve().parent

# 2. Inserta esa ruta al principio de sys.path
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# 3. Ahora ya puedes importar
from variables import *

BIO_CSV_PATH = 'biografias_raw_corrected.csv'
FREQ_CSV_PATH = '../Capítulo primero/frecuentes_corrected.csv'

In [56]:
# Asumimos que el CSV biografias_raw.csv ya tiene encabezado en la primera fila:
df_bio = pd.read_csv(BIO_CSV_PATH, encoding='utf-8', quoting=csv.QUOTE_ALL)

# Leer el CSV de información adicional de los personajes
df_freq = pd.read_csv(FREQ_CSV_PATH, encoding='utf-8').drop(columns=["Frequency"])

# Identificar columnas de modelos (todas menos "Name")
model_columns = [col for col in df_bio.columns if col.lower() != "name"]

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('vader_lexicon')

# 5. Inicializar el analizador VADER
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [57]:
# Función auxiliar para obtener top-N keywords de un vector TF-IDF
def get_top_n_keywords(row_tfidf, feature_names, n=5):
	# row_tfidf es un vector escaso (sparse) de TF-IDF para un documento
	row_array = row_tfidf.toarray().flatten()
	if np.all(row_array == 0):
		return []
	top_n_idx = row_array.argsort()[-n:][::-1]
	return feature_names[top_n_idx].tolist()

# 5. Función para sanitizar nombres de modelo y que sean válidos como filenames
def sanitize_model_name(name: str) -> str:
    # Reemplaza cualquier caracter no alfanumérico (excepto guiones bajos o puntos) por guión bajo
    safe = re.sub(r"[^A-Za-z0-9\-_\.]", "_", name)
    return safe

In [58]:
for model_name in model_columns:
    
	# 7.1. Sanitizamos el nombre para el CSV de salida
    modelo_seguro = sanitize_model_name(model_name)
    
    # 7.2. Comprobamos que exista esa columna en df_bio
    if model_name not in df_bio.columns:
        print(f">> Advertencia: la columna '{model_name}' no existe en biografias_raw.csv. Se salta este modelo.")
        continue
    
    # Extraer solo la columna de nombre y la columna de ese modelo
    temp = df_bio[["Name", model_name]].copy()
    temp = temp.rename(columns={model_name: "biography"})
    
    # Preprocesamiento mínimo: rellenar nulos con cadena vacía
    temp["biography"] = temp["biography"].fillna("")

    # --------------- TF-IDF para extraer Top-5 palabras clave ---------------
    # Construimos un corpus con todas las biografías de este modelo
    corpus = temp["biography"].tolist()
    vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,1))
    tfidf_matrix = vectorizer.fit_transform(corpus)  # matriz (n_docs x n_vocab)
    feature_names = np.array(vectorizer.get_feature_names_out())
    
	# Contenedor para los resultados
    results = {
        "Name": [],
        "top5_keywords": [],
        "sent_neg": [],
        "sent_neu": [],
        "sent_pos": [],
        "sent_compound": [],
        "polarity": [],
        "subjectivity": []
    }
    
	# Iterar por cada fila/documento
    for idx, row in temp.iterrows():
        name = row["Name"]
        text = row["biography"]
        
        # Extraer Top-5 keywords
        tfidf_vector = tfidf_matrix[idx]
        top5 = get_top_n_keywords(tfidf_vector, feature_names, n=5)
        
        # Sentimiento con VADER
        vs = sia.polarity_scores(text)
        # vs devuelve diccionario con keys ['neg','neu','pos','compound']
        
        # Polarity y subjectivity con TextBlob
        tb = TextBlob(text)
        pol = tb.sentiment.polarity
        subj= tb.sentiment.subjectivity
        
        # Guardar en listas
        results["Name"].append(name)
        results["top5_keywords"].append(", ".join(top5))   # guardamos como string separado por comas
        results["sent_neg"].append(vs["neg"])
        results["sent_neu"].append(vs["neu"])
        results["sent_pos"].append(vs["pos"])
        results["sent_compound"].append(vs["compound"])
        results["polarity"].append(pol)
        results["subjectivity"].append(subj)
    
    # Crear DataFrame de resultados de este modelo
    df_results = pd.DataFrame(results)
    df_merged = pd.merge(df_results, df_freq, on="Name", how="left")
    
	# reordenar columnas
    demographic_cols = [c for c in df_freq.columns if c != "Name"]
    metric_cols      = [
        "top5_keywords",
        "sent_neg",
        "sent_neu",
        "sent_pos",
        "sent_compound",
        "polarity",
        "subjectivity"
    ]
    new_column_order = ["Name"] + demographic_cols + metric_cols
    df_merged = df_merged[new_column_order]
	
    # Unir con la info demográfica de df_freq (opcional, pero útil para ver sesgos por género, país, etc.)
    # Nota: asumimos que df_freq tiene una columna "Name" compatible
    
    fname = f"{modelo_seguro}.csv"
    df_merged.to_csv(fname, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    
    print(f">> Resultados para modelo '{model_name}' guardados en: {fname}")

>> Resultados para modelo 'openai/gpt-4o-mini' guardados en: openai_gpt-4o-mini.csv
>> Resultados para modelo 'deepseek/deepseek-chat-v3-0324' guardados en: deepseek_deepseek-chat-v3-0324.csv
>> Resultados para modelo 'google/gemini-2.0-flash-001' guardados en: google_gemini-2.0-flash-001.csv
>> Resultados para modelo 'microsoft/phi-4-multimodal-instruct' guardados en: microsoft_phi-4-multimodal-instruct.csv
>> Resultados para modelo 'meta-llama/llama-4-maverick' guardados en: meta-llama_llama-4-maverick.csv
